In [17]:
import datetime  #
import os.path  # 路径管理
import sys  # 获取当前运行脚本的路径 (in argv[0])

import backtrader as bt

# 创建策略继承bt.Strategy
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        # 记录策略的执行日志
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # 保存收盘价的引用
        self.dataclose = self.data_close
        print('切片访问前3个值', self.datas[0].array[:3])
        print('close0',self.dataclose[0],'close1',self.dataclose[1])

    def next(self):
        # 记录收盘价
        self.log('Close, %.2f' % self.dataclose[0])
        pass


if __name__ == '__main__':
    # 创建Cerebro引擎
    cerebro = bt.Cerebro()
    # Cerebro引擎在后台创建broker(经纪人)，系统默认资金量为10000

    # 为Cerebro引擎添加策略
    cerebro.addstrategy(TestStrategy)

    # 获取当前运行脚本所在目录 modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    modpath = os.path.abspath('.')
    # 拼接加载路径
    datapath = os.path.join(modpath, '../../datas/orcl-1995-2014.txt')

    # 创建交易数据集
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        # 数据必须大于fromdate 1995-01-03
        fromdate=datetime.datetime(1995, 1, 3),
        # 数据必须小于todate
        todate=datetime.datetime(1995, 1, 30),
        reverse=False)


    # 加载交易数据
    cerebro.adddata(data)

    # Handy dictionary for the argument timeframe conversion
    tframes = dict(
        daily=bt.TimeFrame.Days,
        weekly=bt.TimeFrame.Weeks,
        monthly=bt.TimeFrame.Months)
    # Add the resample data instead of the original
    data1 = cerebro.resampledata(data,
                         timeframe=tframes['weekly'],
                         compression=1)
    # 加载交易数据
    # cerebro.adddata(data1)

    # 设置投资金额100000.0
    cerebro.broker.setcash(100000.0)
    # 引擎运行前打印期出资金
    print('组合期初资金: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    # 引擎运行后打期末资金
    print('组合期末资金: %.2f' % cerebro.broker.getvalue())

组合期初资金: 100000.00
切片访问前3个值 array('d')


IndexError: array index out of range